In [1]:
import pandas as pd
from datetime import datetime


In [7]:
def process_date(date_string_series):
    formatted_date_series = pd.to_datetime(date_string_series, format="%m/%d/%Y").dt.strftime("%Y年%m月%d日")
    return formatted_date_series

In [2]:
df_input = pd.read_csv('transactions-2.csv')

In [3]:
columns_exp = ['日期', '类型', '金额', '一级分类', '二级分类', '账户1', '账户2', '备注', '货币', '标签']

In [4]:
columns_TD = ["Date", "Description", "Original Description", "Amount", "Transaction Type", "Category", "Account Name", "Labels", "Notes"]

In [20]:
account_mapping = {"TD CASH BACK VISA* CARD":308, "TD EVERY DAY SAVINGS ACCOUNT":995, "TD UNLIMITED CHEQUING ACCOUNT":995 }
transaction_mapping = {"Income":"收入", "Interest Income":"收入",}
category_mapping ={   
'FRESHCO': ['餐饮'],
'APPLE': ['数码', '软件'],
'LUCY':['餐饮', '三餐'],
'SHOPPERS DRUG':['购物'],
'MCDONALD':['餐饮', '三餐'],
'MARSHALLS/HOMES':['购物','日用'],
'MEAL':['餐饮', '三餐'],
'FIDO MOBILE BPY':['通讯'],
'WESTERN DRUG MART':['购物'],
'IVEY BUSINESS SCHOOL': ['餐饮', '三餐'],
'SEPHORA': ['护肤'],
'Temu': ['购物'],
'URBAN PLANET':['购物'],
'COCA COLA': ['餐饮','零食'],
}

In [5]:
df_exp = pd.DataFrame(columns=columns_exp)

In [8]:
df_exp['日期'] = process_date(df_input['Date'])

In [36]:
df_exp['备注'] = df_input['Description']
df_exp['金额'] = df_input['Amount']
df_exp['账户1'] = df_input['Account Name'].map(account_mapping)
df_exp['类型'] = df_input['Category'].map(transaction_mapping)
df_exp['类型'].fillna("支出", inplace=True)
df_exp['货币'] = 'CAD'
df_exp['一级分类'] = df_input['Description'].apply(lambda x: next((category_mapping[key][0] for key in category_mapping.keys() if key in x), None))
df_exp['二级分类'] = df_input['Description'].apply(lambda x: next((category_mapping[key][1] for key in category_mapping.keys() if key in x and len(category_mapping[key]) > 1), None))

In [35]:
df_exp.to_csv('e.csv')

In [27]:
df_exp

,日期,类型,金额,一级分类,二级分类,账户1,账户2,备注,货币,标签
0,2023年11月07日,支出,19.15,购物,[购物],308,NaN,WESTERN DRUG MART,CAD,NaN
1,2023年11月06日,支出,1000.00,None,None,995,NaN,UJ534 TFR-TO 6258995,CAD,NaN
2,2023年11月06日,支出,1074.50,None,None,995,NaN,View more SEND E-TFR ***zMR View more SEND E-T...,CAD,NaN
3,2023年11月06日,收入,1000.00,None,None,995,NaN,UJ534 TFR-FR 6258588,CAD,NaN
4,2023年10月31日,收入,0.14,None,None,995,NaN,INTEREST CREDIT,CAD,NaN
...,...,...,...,...,...,...,...,...,...,...
63,2023年09月11日,支出,0.00,None,None,995,NaN,OPEN ACCOUNT,CAD,NaN
64,2023年09月11日,支出,820.00,None,None,995,NaN,View more SEND E-TFR ***GzW View more SEND E-T...,CAD,NaN
65,2023年09月11日,支出,2000.00,None,None,308,NaN,PAYMENT - THANK YOU,CAD,NaN
66,2023年09月11日,收入,16000.00,None,None,995,NaN,RJ024 TFR-FR 6258995,CAD,NaN
